In [ ]:
import requestsa
from bs4 import BeautifulSoup

bird_name = "Acacia Pied Barbet"
search_url = f"https://app.birda.org/species-guide/search?q={bird_name.replace(' ', '+')}"
response = requests.get(search_url, headers={"User-Agent": "Mozilla/5.0"})
response.status_code

500

In [30]:
bird_name.replace(' ', '+')

'Acacia+Pied+Barbet'

In [24]:
soup

<!DOCTYPE html>

<html lang="en-US" prefix="og: https://ogp.me/ns#">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://gmpg.org/xfn/11" rel="profile"/>
<style id="abst-dynamic-hide">body:not(.abst-show-page).page-id-130,body:not(.abst-show-page).postid-130{opacity:0;pointer-events:none;}body.abst-show-page{display:revert!important;}</style><noscript><style>body:not(.abst-show-page).page-id-130,body:not(.abst-show-page).postid-130{opacity:1;pointer-events:auto;}</style></noscript>
<!-- Search Engine Optimization by Rank Math - https://rankmath.com/ -->
<title>Page Not Found | Birda</title>
<meta content="follow, noindex" name="robots">
<meta content="en_US" property="og:locale">
<meta content="article" property="og:type"/>
<meta content="Page Not Found | Birda" property="og:title"/>
<meta content="Birda" property="og:site_name"/>
<meta content="https://www.facebook.com/birda.org/" property="article:publisher"/>
<meta

In [25]:
import requests
from bs4 import BeautifulSoup
import json
import re

def extract_birda_data_fixed(species_slug):
    # https://app.birda.org/species-guide/11404/Acacia_Pied_Barbet
    bird_url = f"https://birda.org/species/{species_slug}"
    
    # Birda requires a real browser header to serve the full page content
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }

    try:
        response = requests.get(bird_url, headers=headers, timeout=20)
        soup = BeautifulSoup(response.text, 'html.parser')

        # 1. Image Extraction (targeting OpenGraph tags first)
        image_url = "Not Found"
        og_tag = soup.find("meta", property="og:image")
        if og_tag:
            image_url = og_tag.get("content")
        
        # 2. Summary Extraction
        # Birda typically places the description in a section or a paragraph near the top
        summary = "No summary found."
        # Attempt to find by common class patterns Birda uses
        desc_container = soup.find("div", class_=re.compile(r"description|about|info", re.I))
        if desc_container:
            summary = desc_container.get_text(separator=" ", strip=True)
        
        # Fallback: Check JSON-LD script (this is where Birda hides most data)
        if summary == "No summary found.":
            script = soup.find("script", type="application/ld+json")
            if script:
                try:
                    data = json.loads(script.string)
                    # JSON-LD can be a list or a single dict
                    main_entity = data[0] if isinstance(data, list) else data
                    summary = main_entity.get("description", "No summary found.")
                except: pass

        # 3. Audio Extraction (vocalizationTypes)
        audio_recordings = []
        # Birda often uses <audio> tags or <a> links to media files
        # We look for the 'src' in audio tags or links ending in .mp3
        media_elements = soup.find_all(['audio', 'source', 'a'])
        
        for elem in media_elements:
            src = elem.get('src') or elem.get('href')
            if src and any(ext in src.lower() for ext in ['.mp3', '.wav', '.ogg']):
                # Clean URL (sometimes they are relative)
                full_src = src if src.startswith('http') else f"https://birda.org{src}"
                
                # Try to guess the type from the context or filename
                rec_type = "Call"
                if "song" in full_src.lower(): rec_type = "Song"
                elif "alarm" in full_src.lower(): rec_type = "Alarm"
                elif "wing" in full_src.lower() or "clap" in full_src.lower(): rec_type = "Mechanical/Sonn"
                
                audio_recordings.append({
                    "type": rec_type,
                    "url": full_src
                })

        # Remove duplicate audio URLs
        seen = set()
        unique_audio = []
        for a in audio_recordings:
            if a['url'] not in seen:
                unique_audio.append(a)
                seen.add(a['url'])

        return {
            "bird_url": bird_url,
            "common_name": species_slug.replace("-", " ").title(),
            "image_url": image_url,
            "summary": summary,
            "audio_recordings": unique_audio
        }

    except Exception as e:
        return {"error": str(e)}

# --- EXECUTION ---
result = extract_birda_data_fixed("Acacia_Pied_Barbet")
print(json.dumps(result, indent=4))

{
    "bird_url": "https://birda.org/species/Acacia_Pied_Barbet",
    "common_name": "Acacia_Pied_Barbet",
    "image_url": "Not Found",
    "summary": "No summary found.",
    "audio_recordings": []
}


In [ ]:
# --- TEST ---
result = extract_birda_data_refined("flappet-lark")
print(json.dumps(result, indent=4))

{
    "error": "Tag.find() got multiple values for argument 'name'"
}
